In [1]:
import os
import ast
import sys
src_dir = os.path.join('..', 'src')
sys.path.append(os.path.abspath(src_dir))

import requests
import json
from os import path as osp
from datetime import datetime, timedelta
import praw

from security import KeyHandler
from data import path

VERSION = 'PedAPI/0.0.3'


class RedditClient:
    def __init__(self, verbose: bool = True):
        self.verbose_print = print if verbose else lambda *a, **k: None
        self.save_path = path("comments")

        self.session_time_out = datetime.now()
        self.kh = KeyHandler()
        self._login()
        self.save_path = path("comments")

    def _login(self):
        authorization = requests.auth.HTTPBasicAuth(
            self.kh.client_id,
            self.kh.secret_key)

        self.kh.headers['User-Agent'] = VERSION
        res = requests.post('https://www.reddit.com/api/v1/access_token',
                            auth=authorization,
                            data=self.kh.login_data,
                            headers=self.kh.headers)
        self.session_time_out = datetime.now() + timedelta(seconds=res.json()['expires_in'] - 60)

        token = res.json()['access_token']
        self.kh.headers['Authorization'] = f'bearer {token}'

        assert self._test_authentication()
        self.verbose_print('Successfully logged as {}'.format(self.kh.login_data['username']))
        self.verbose_print('Session expires on {}'.format(str(self.session_time_out)))

    def _test_authentication(self):
        if requests.get('https://oautch.reddit.com/api/v1/me', headers=self.kh.headers).status_code == 200:
            return True
        return False

    def _manage_session_time_out(self):
        if datetime.now() > self.session_time_out:
            self.verbose_print('Renewing session...')
            self._login()

    def get_comments(self, post_id: str):
        self._manage_session_time_out()

        link = 'https://oauth.reddit.com/r/wallstreetbets/comments/'
        res = requests.get(link + post_id, headers=self.kh.headers)

        with open(osp.join(self.save_path, post_id + '.json'), 'w') as outfile:
            json.dump(res.json(), outfile, indent=4)


class Praw:
    def __init__(self, verbose: bool = True):
        self.verbose_print = print if verbose else lambda *a, **k: None

        self.kh = KeyHandler()
        self._login()

    def _login(self):
        self.reddit = praw.Reddit(
            client_id=self.kh.client_id,
            client_secret=self.kh.secret_key,
            user_agent=VERSION,
            username=self.kh.login_data['username'],
            password=self.kh.login_data['password']
        )

    def get_submission(self, id_):
        return self.reddit.submission(id=id_)


In [2]:
client = Praw()

In [3]:
submission = client.get_submission('nrblju')

In [4]:
for i, top_level_comment in enumerate(submission.comments):
    while True:
        try:
            submission.comments.replace_more(None)
            break
        except:
            print("Handling replace_more exception")
            sleep(1)
            
    print(i, top_level_comment.body, top_level_comment.created_utc)

0 Main mods I'm using are More Archotech Garbage, Science never stops AOTC, Rimatomics, Rimfactory, Misc Robots, VFE mechs plus a bunch of OP enemy factions to try (and fail) to balance the AOTC stuff a bit.

I've retextured the Cosmic reactors from AOTC as well to fit in more with the Archotech style of the base.  


Also I wouldn't suggest doing a circle style base, it is ridiculously resource and work intensive, does look pretty cool though. 1622721313.0
1 Ah, yes, the Arcotech and glitter tech "hell hole". Let's be clear, if you offer me a ride off of this base, I'll stay on the base. Pretty sure it's self sufficient at this point. 1622728569.0
2 I don't get how people can make such aesthetic bases. It's utilitarian rectangles all the way for me... 1622740894.0
3 A circular kill box... intriguing. 1622722668.0
4 Round AND mountain? Now this is quite the rare sight, most mountain bases on here are boxy and aren’t such an interesting shape. Good job. 1622722889.0
5 Off to crash somew

In [5]:
submission.num_comments

54

In [6]:
wanted = set(['all_awardings', 'body', 'created', 'created_utc', 'depth', 'downs', 'id', 'parent_id', 'replies', 'score', 'ups', 'data', 'kind', 'children', 'name'])

In [7]:
import pandas as pd
import numpy as np

def replace_more(submission):
    while True:
        try:
            submission.comments.replace_more()
            break
        except:
            print("Handling replace_more exception")
            sleep(1)

def obtain_comments_for_id(praw_client, submission_id):
    submission = praw_client.get_submission(submission_id)

    replace_more(submission)
    
    res = [obtain_comments(comment, submission_id) for comment in list(submission.comments)]
    df = pd.concat(res, ignore_index=True)
    df.columns = ['id', 'parent_id', 'post_id', 'body', 'created_utc', 'depth', 'score', 'ups', 'downs']
    return df
    

def obtain_comments(comment, submission_id):

    item = [comment.id, comment.parent_id[3:], submission_id, comment.body, comment.created_utc, comment.depth, comment.score, comment.ups, comment.downs]
        
    replies = []
    for reply in list(comment.replies):
        replies.extend([obtain_comments(reply, submission_id)])
    
    return pd.concat([pd.DataFrame(np.array([item]))] + replies)

In [8]:
obtain_comments_for_id(client, 'nrblju')

,id,parent_id,post_id,body,created_utc,depth,score,ups,downs
0,h0fg6t4,nrblju,nrblju,Main mods I'm using are More Archotech Garbage...,1622721313.0,0,44,44,0
1,h0gj95t,h0fg6t4,nrblju,"Why circles? Only my warehouses, stables and g...",1622738934.0,1,12,12,0
2,h0h7rr1,h0gj95t,nrblju,I've just done a lot of square boxy bases with...,1622749153.0,2,13,13,0
3,h0hzba0,h0gj95t,nrblju,Why corridors 3 tiles wide? I never got proble...,1622761158.0,2,3,3,0
4,h0hztbs,h0hzba0,nrblju,You can cram more defenders in wider corridors...,1622761403.0,3,6,6,0
5,h0in1y6,h0hztbs,nrblju,I always tell myself that it's for practical r...,1622772776.0,4,3,3,0
6,h0jmvl4,h0in1y6,nrblju,Isnt walking though 1/2-wide corridor slower?,1622795687.0,5,3,3,0
7,h0k0866,h0hztbs,nrblju,"But they can get more attackers in, and usuall...",1622806640.0,4,1,1,0
8,h0kb7wq,h0k0866,nrblju,You can put melee-blocking chokepoints into 3-...,1622812531.0,5,2,2,0
9,h0k6c63,h0k0866,nrblju,"It’s from over function really, there are defi...",1622810024.0,5,1,1,0


In [9]:
def identity(x: str) -> str:
    return x

In [10]:
identity(12)

12

In [12]:
from data import save_dataset, get_dataset
from num_comments import sample_submissions

In [13]:
data = get_dataset('num_comments.csv')

../../data/datasets/num_comments.csv


In [14]:
sampled = sample_submissions(data)
sampled

,id,title,score,url,comms_num,created,body,timestamp,upvote_ratio,is_oc,...,body_stem_tokens,image_hue,image_sat,image_val,image_label,gme_price,gme_volume,ocr_text,ocr_text_word_count,num_comments
10665,l6b909,🚨🚨FAKE NEWS🚨🚨,11,https://i.redd.it/fep2zut65xd61.jpg,0,1.611772e+09,NaN,Wed Jan 27 19:32:30 2021,0.91,False,...,['nan'],0.490841,0.490841,0.490841,television,345.329987,7113878,"['option', 'activity', 'gamestop', 'open', 'in...",42,0
15864,l6yomd,RH just killed SNDL,9,https://www.reddit.com/r/wallstreetbets/commen...,0,1.611846e+09,Bruh,Thu Jan 28 15:54:41 2021,0.86,False,...,['bruh'],-1.000000,-1.000000,-1.000000,NaN,483.000000,18660587,[],0,0
23766,l72903,Dang,6,https://www.reddit.com/r/wallstreetbets/commen...,0,1.611853e+09,They are really laying it on us today. I'm hol...,Thu Jan 28 17:58:17 2021,0.88,False,...,"['today', 'ride', 'ill', 'u', 'hold', 'really'...",-1.000000,-1.000000,-1.000000,NaN,324.000000,10238490,[],0,0
15537,l6yk56,It's ONLY EXTERNAL Brokerage Firms Blocked (Al...,4,https://www.reddit.com/r/wallstreetbets/commen...,0,1.611845e+09,I can still trade on Charles Schwab & Meryl Ly...,Thu Jan 28 15:50:27 2021,1.00,False,...,"['amc', 'town', 'let', 'nok', 'gme', 'goooo', ...",-1.000000,-1.000000,-1.000000,NaN,483.000000,18660587,[],0,0
33638,lb7bg4,Full Retard,1,https://www.reddit.com/r/wallstreetbets/commen...,0,1.612302e+09,Everyone knows the famous quote from black-fac...,Tue Feb 2 22:40:54 2021,1.00,False,...,"['never', 'gme', 'know', 'well', 'speed', 'fam...",-1.000000,-1.000000,-1.000000,NaN,98.500000,3577648,[],0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38779,lgvkn4,Ford Motors (F) DD,235,https://www.reddit.com/r/wallstreetbets/commen...,184,1.612970e+09,**Overview** Ford Motors is currently underva...,Wed Feb 10 16:08:17 2021,0.80,False,...,"['company', 'rivian', 'argue', 'ipo', 'go', 'k...",-1.000000,-1.000000,-1.000000,NaN,53.759998,3260725,[],0,185
35088,lbtzse,Prepare for take off enjoy the ride,4128,https://www.reddit.com/r/wallstreetbets/commen...,508,1.612376e+09,"Ladies and gentlemen, welcome onboard Flight A...",Wed Feb 3 19:12:49 2021,0.93,False,...,"['also', 'prohibit', 'device', 'expect', 'tray...",-1.000000,-1.000000,-1.000000,NaN,109.598297,4997392,[],0,508
33,l0lg6r,Shitron Attacking begins.,23525,https://i.redd.it/n43f5b8szac61.jpg,2638,1.611068e+09,NaN,Tue Jan 19 15:58:53 2021,0.89,False,...,['nan'],0.085924,0.085924,0.085924,web_site,45.520000,34966133,"['citron', 'research', 'citron', 'research', '...",38,2614
25289,l785l4,This is WAR $GME $BB $NOK,2767,https://www.reddit.com/r/wallstreetbets/commen...,398,1.611866e+09,One thing everyone here must understand: the a...,Thu Jan 28 21:41:26 2021,0.98,False,...,"['benefit', 'freedom', 'matter', 'outcome', 'g...",-1.000000,-1.000000,-1.000000,NaN,237.990005,3520581,[],0,395


In [23]:
cut = sampled[sampled.num_comments < sampled.num_comments.quantile(0.99)].copy()
cut.sort_values(by=['num_comments'], inplace=True)
cut

,id,title,score,url,comms_num,created,body,timestamp,upvote_ratio,is_oc,...,body_stem_tokens,image_hue,image_sat,image_val,image_label,gme_price,gme_volume,ocr_text,ocr_text_word_count,num_comments
10665,l6b909,🚨🚨FAKE NEWS🚨🚨,11,https://i.redd.it/fep2zut65xd61.jpg,0,1.611772e+09,NaN,Wed Jan 27 19:32:30 2021,0.91,False,...,['nan'],0.490841,0.490841,0.490841,television,345.329987,7113878,"['option', 'activity', 'gamestop', 'open', 'in...",42,0
7067,l695ss,$EXPR,1,https://www.reddit.com/r/wallstreetbets/commen...,0,1.611767e+09,Bought and holding $EXPR.,Wed Jan 27 18:07:36 2021,0.54,False,...,"['buy', 'hold', 'expr']",-1.000000,-1.000000,-1.000000,NaN,349.989990,7610641,[],0,0
33863,lb7j6y,How are there more BUY than SELL order for tod...,1,https://www.reddit.com/r/wallstreetbets/commen...,0,1.612302e+09,Could someone explain this to me like I have h...,Tue Feb 2 22:48:19 2021,1.00,False,...,"['brain', 'like', 'someone', 'process', 'img',...",-1.000000,-1.000000,-1.000000,NaN,98.500000,3577648,[],0,0
5394,l68joi,$VIRT,0,https://i.redd.it/zjgxq36klwd61.png,0,1.611766e+09,NaN,Wed Jan 27 17:42:30 2021,0.44,False,...,['nan'],0.153664,0.153664,0.153664,buckle,380.000000,11609057,[],0,0
17731,l6zgup,"IF YOU’RE SELLING $KOSS, YOUR WIFE’S BOYFRIEND...",7,https://www.reddit.com/r/wallstreetbets/commen...,0,1.611847e+09,NaN,Thu Jan 28 16:20:45 2021,0.82,False,...,['nan'],-1.000000,-1.000000,-1.000000,NaN,403.999908,7547213,[],0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1500,l30tps,Boomer Old Money has bled and will get dirty g...,21487,https://www.reddit.com/r/wallstreetbets/commen...,1573,1.611361e+09,🚨🚨🚨Alert add - https://www.bloomberg.com/news/...,Sat Jan 23 01:15:29 2021,0.92,False,...,"['smart', 'within', 'brag', 'gain', 'count', '...",-1.000000,-1.000000,-1.000000,NaN,68.000000,18687884,[],0,1561
25319,l78vtv,"strap in boys, they're coming",43698,https://i.redd.it/yticvqhh15e61.png,1597,1.611868e+09,NaN,Thu Jan 28 22:07:55 2021,0.98,False,...,['nan'],0.525229,0.525229,0.525229,web_site,237.990005,3520581,"['joe', 'mccann', 'ajoemccann', 'fox', 'news',...",48,1585
25641,l7f95j,"Ladies and Gentlemen, WE GOT THEM",82030,https://v.redd.it/bva2q98dc6e61,1630,1.611884e+09,NaN,Fri Jan 29 02:29:26 2021,0.94,False,...,['nan'],-1.000000,-1.000000,-1.000000,NaN,237.990005,3520581,[],0,1621
26423,l82qwt,WE HAVE ALMOST WON OUR FIRST MAJOR BATTLE 💎🙌💎🙌...,25375,https://www.reddit.com/r/wallstreetbets/commen...,1661,1.611951e+09,AS MANY OF YOU KNOW THERE ARE CONTRACTS THAT E...,Fri Jan 29 21:09:52 2021,0.98,False,...,"['today', 'let', 'overall', 'edit', 'hedgefund...",-1.000000,-1.000000,-1.000000,NaN,335.000000,5085871,[],0,1641


In [26]:
cut.num_comments[:7500].sum()

163777

In [34]:
def obtain_comments_for_id(praw_client, submission_id):
    starttime = time()
    print("Obtaining comments for submission '{}'...".format(submission_id), end='', flush=True)
    
    submission = praw_client.get_submission(submission_id)

    replace_more(submission)
    
    res = [obtain_comments(comment, submission_id) for comment in list(submission.comments)]
    
    if len(res) > 0:
        df = pd.concat(res, ignore_index=True)
        print(" [{} comments]".format(df.shape[0]), end='', flush=True)
    else:
        no_comments = [['', '', submission_id, '', 0, -1, 0, 0, 0]]
        df = pd.DataFrame(np.array(no_comments))
        print(" [no comments]", end='', flush=True)

    df.columns = ['id', 'parent_id', 'post_id', 'body', 'created_utc', 'depth', 'score', 'ups', 'downs']
    print("   finished in {} sec".format(np.around(time()-starttime,2)))
    return df

In [40]:
from tqdm.notebook import tqdm

df = obtain_comments_for_id(client, 'l6b909')

i = 0
for subm_id in tqdm(cut.id[1:7500]):
    i += 1
    
    df = df.append(obtain_comments_for_id(client, subm_id))
    
    if i % 100 == 0:
        print("Autosaving {}".format(i))
        save_dataset(df, 'new_comments_autosave_{}.csv'.format(i))

save_dataset(df, 'new_comments_all.csv')

Obtaining comments for submission 'l6b909'... [no comments]   finished in 0.73 sec


  0%|          | 0/7499 [00:00<?, ?it/s]

Obtaining comments for submission 'l695ss'... [no comments]   finished in 0.24 sec
Obtaining comments for submission 'lb7j6y'... [no comments]   finished in 0.22 sec
Obtaining comments for submission 'l68joi'... [no comments]   finished in 0.28 sec
Obtaining comments for submission 'l6zgup'... [no comments]   finished in 0.24 sec
Obtaining comments for submission 'l71tul'... [no comments]   finished in 0.27 sec
Obtaining comments for submission 'l6zkp2'... [no comments]   finished in 0.24 sec
Obtaining comments for submission 'l6a0h9'... [no comments]   finished in 0.3 sec
Obtaining comments for submission 'l70qgd'... [no comments]   finished in 0.43 sec
Obtaining comments for submission 'l6xakb'... [no comments]   finished in 0.22 sec
Obtaining comments for submission 'l6ylyh'... [no comments]   finished in 0.22 sec
Obtaining comments for submission 'l6a9rd'... [no comments]   finished in 0.31 sec
Obtaining comments for submission 'l6wq99'... [no comments]   finished in 0.22 sec
Obtai

Obtaining comments for submission 'l74lcg'... [no comments]   finished in 1.2 sec
Autosaving 100
Obtaining comments for submission 'l71v4y'... [no comments]   finished in 1.2 sec
Obtaining comments for submission 'l7199v'... [no comments]   finished in 0.75 sec
Obtaining comments for submission 'lb6xh4'... [no comments]   finished in 0.78 sec
Obtaining comments for submission 'l688oy'... [no comments]   finished in 1.24 sec
Obtaining comments for submission 'lb76md'... [no comments]   finished in 0.72 sec
Obtaining comments for submission 'l68v4k'... [no comments]   finished in 1.22 sec
Obtaining comments for submission 'l727g0'... [no comments]   finished in 0.73 sec
Obtaining comments for submission 'l68sfl'... [no comments]   finished in 1.27 sec
Obtaining comments for submission 'l6yn0a'... [no comments]   finished in 0.74 sec
Obtaining comments for submission 'l6yjkf'... [no comments]   finished in 1.2 sec
Obtaining comments for submission 'lb70n8'... [no comments]   finished in 0

Obtaining comments for submission 'lcjbg4'... [no comments]   finished in 0.77 sec
Obtaining comments for submission 'l71xnf'... [no comments]   finished in 1.25 sec
Obtaining comments for submission 'l6zz2b'... [no comments]   finished in 0.73 sec
Autosaving 200
Obtaining comments for submission 'lb9gmx'... [no comments]   finished in 3.23 sec
Obtaining comments for submission 'l6yftj'... [no comments]   finished in 0.74 sec
Obtaining comments for submission 'l6yskr'... [no comments]   finished in 0.22 sec
Obtaining comments for submission 'l6zeaj'... [no comments]   finished in 0.28 sec
Obtaining comments for submission 'l69ulc'... [no comments]   finished in 0.7 sec
Obtaining comments for submission 'l706e4'... [no comments]   finished in 0.7 sec
Obtaining comments for submission 'l6949g'... [no comments]   finished in 1.24 sec
Obtaining comments for submission 'l6yguc'... [no comments]   finished in 0.78 sec
Obtaining comments for submission 'l68njg'... [no comments]   finished in 

...

Obtaining comments for submission 'lc43uq'... [42 comments]   finished in 1.01 sec
Obtaining comments for submission 'l52vlb'... [80 comments]   finished in 1.26 sec
Obtaining comments for submission 'law52t'... [58 comments]   finished in 1.2 sec
Obtaining comments for submission 'lcg86d'... [68 comments]   finished in 1.15 sec
Obtaining comments for submission 'lbzvxx'... [22 comments]   finished in 0.54 sec
Obtaining comments for submission 'l7ai3u'... [70 comments]   finished in 1.16 sec
Obtaining comments for submission 'laaz15'... [27 comments]   finished in 0.93 sec
Obtaining comments for submission 'l3kkgs'... [87 comments]   finished in 1.27 sec
Obtaining comments for submission 'l9gds5'... [90 comments]   finished in 1.37 sec
Obtaining comments for submission 'l9vx4t'... [51 comments]   finished in 0.66 sec
Obtaining comments for submission 'l7mep6'... [67 comments]   finished in 0.71 sec
Obtaining comments for submission 'la7sy5'... [34 comments]   finished in 1.02 sec
Obtai

Obtaining comments for submission 'ladtdw'... [32 comments]   finished in 0.52 sec
Obtaining comments for submission 'lb0qkj'... [75 comments]   finished in 1.34 sec
Obtaining comments for submission 'l9ym89'... [19 comments]   finished in 1.02 sec
Obtaining comments for submission 'l8aldr'... [102 comments]   finished in 1.74 sec
Obtaining comments for submission 'lbaj2p'... [30 comments]   finished in 0.77 sec
Obtaining comments for submission 'lasuet'... [31 comments]   finished in 1.28 sec
Obtaining comments for submission 'l7ui8i'... [82 comments]   finished in 1.09 sec
Obtaining comments for submission 'lgtdtj'... [68 comments]   finished in 1.57 sec
Obtaining comments for submission 'l57xsx'... [99 comments]   finished in 1.47 sec
Obtaining comments for submission 'l7k3f6'... [86 comments]   finished in 1.35 sec
Obtaining comments for submission 'l53uj4'... [102 comments]   finished in 0.94 sec
Obtaining comments for submission 'lanw94'... [18 comments]   finished in 0.75 sec
Ob

Obtaining comments for submission 'l7h8hv'... [110 comments]   finished in 1.41 sec
Obtaining comments for submission 'l4wmkl'... [120 comments]   finished in 1.33 sec
Obtaining comments for submission 'l8ujpp'... [113 comments]   finished in 1.51 sec
Obtaining comments for submission 'lahvcz'... [45 comments]   finished in 1.09 sec
Obtaining comments for submission 'lbv6iy'... [61 comments]   finished in 0.75 sec
Obtaining comments for submission 'l567ag'... [107 comments]   finished in 1.15 sec
Obtaining comments for submission 'laij9o'... [51 comments]   finished in 1.35 sec
Obtaining comments for submission 'l9f2wn'... [113 comments]   finished in 1.09 sec
Obtaining comments for submission 'lbquft'... [45 comments]   finished in 0.7 sec
Obtaining comments for submission 'l7h5co'... [89 comments]   finished in 1.02 sec
Obtaining comments for submission 'l7hra2'... [95 comments]   finished in 1.06 sec
Obtaining comments for submission 'labnf8'... [35 comments]   finished in 0.82 sec


Obtaining comments for submission 'l6ruv8'... [96 comments]   finished in 1.53 sec
Obtaining comments for submission 'l8l0xy'... [134 comments]   finished in 1.63 sec
Obtaining comments for submission 'l8kbgq'... [144 comments]   finished in 1.45 sec
Obtaining comments for submission 'l85rgr'... [136 comments]   finished in 1.13 sec
Obtaining comments for submission 'l8bz7j'... [134 comments]   finished in 1.77 sec
Obtaining comments for submission 'l8a022'... [133 comments]   finished in 3.09 sec
Obtaining comments for submission 'l6u9wu'... [65 comments]   finished in 1.54 sec
Obtaining comments for submission 'l9idz3'... [123 comments]   finished in 2.23 sec
Obtaining comments for submission 'l7c2u2'... [99 comments]   finished in 1.05 sec
Obtaining comments for submission 'lfu7mj'... [114 comments]   finished in 1.65 sec
Obtaining comments for submission 'l1n28t'... [129 comments]   finished in 1.12 sec
Obtaining comments for submission 'l9mmvb'... [135 comments]   finished in 1.05

Obtaining comments for submission 'l9mxyo'... [161 comments]   finished in 1.35 sec
Obtaining comments for submission 'l9pg4u'... [141 comments]   finished in 1.17 sec
Obtaining comments for submission 'l6pkzz'... [113 comments]   finished in 1.07 sec
Obtaining comments for submission 'l7gxq0'... [126 comments]   finished in 1.55 sec
Obtaining comments for submission 'l916hu'... [162 comments]   finished in 1.75 sec
Obtaining comments for submission 'l9fs4x'... [136 comments]   finished in 1.21 sec
Obtaining comments for submission 'l9tynq'... [95 comments]   finished in 1.02 sec
Obtaining comments for submission 'l0uba5'... [165 comments]   finished in 1.29 sec
Obtaining comments for submission 'l8g9iv'... [159 comments]   finished in 1.17 sec
Obtaining comments for submission 'lci5t2'... [128 comments]   finished in 1.4 sec
Obtaining comments for submission 'l520gh'... [155 comments]   finished in 1.96 sec
Obtaining comments for submission 'l7pyef'... [135 comments]   finished in 1.0

In [39]:
from time import time

In [41]:
import seaborn as sns

In [42]:
df

,id,parent_id,post_id,body,created_utc,depth,score,ups,downs
0,,,l6b909,,0,-1,0,0,0
0,,,l695ss,,0,-1,0,0,0
0,,,lb7j6y,,0,-1,0,0,0
0,,,l68joi,,0,-1,0,0,0
0,,,l6zgup,,0,-1,0,0,0
...,...,...,...,...,...,...,...,...,...
78,glmtfhq,glmt389,la4vu6,Är du bra på att använda reddit? Hur fan kan j...,1612205599.0,1,1,1,0
79,glnlljh,glmtfhq,la4vu6,"Tyvärr, jag vet inte heller hur man gör. Bruka...",1612216535.0,2,1,1,0
80,glnbutf,la4vu6,la4vu6,I like your AgEagle!,1612212724.0,0,1,1,0
81,glncs3t,la4vu6,la4vu6,First time investor and Just opened a charles ...,1612213055.0,0,1,1,0
